# OrbitalShield Exploratory Data Analysis

In [5]:
# !pip install spacetrack

In [35]:
# !pip install sgp4

In [138]:
from datetime import(
    datetime,
    timedelta,
    timezone,
)
import getpass
import json

import numpy as np
import pandas as pd
from sgp4.api import(
    jday,
    Satrec,
    SatrecArray,
)
from spacetrack import SpaceTrackClient

## Conjunction Assess

In [ ]:
# Define Space-Track credential
ST_USER = input("Username:")
ST_PWD = getpass.getpass("Password:")

In [113]:
# Fetch latest satellite records
with SpaceTrackClient(ST_USER, ST_PWD) as st:
    resp = st.gp(
        epoch=">now-1",
        format="json",
    )

In [ ]:
"""Use metadata from JSON to create satellite dim table"""
# Convert JSON records to dataframe
df = pd.DataFrame(json.loads(resp))

df.columns

Index(['CCSDS_OMM_VERS', 'COMMENT', 'CREATION_DATE', 'ORIGINATOR',
       'OBJECT_NAME', 'OBJECT_ID', 'CENTER_NAME', 'REF_FRAME', 'TIME_SYSTEM',
       'MEAN_ELEMENT_THEORY', 'EPOCH', 'MEAN_MOTION', 'ECCENTRICITY',
       'INCLINATION', 'RA_OF_ASC_NODE', 'ARG_OF_PERICENTER', 'MEAN_ANOMALY',
       'EPHEMERIS_TYPE', 'CLASSIFICATION_TYPE', 'NORAD_CAT_ID',
       'ELEMENT_SET_NO', 'REV_AT_EPOCH', 'BSTAR', 'MEAN_MOTION_DOT',
       'MEAN_MOTION_DDOT', 'SEMIMAJOR_AXIS', 'PERIOD', 'APOAPSIS', 'PERIAPSIS',
       'OBJECT_TYPE', 'RCS_SIZE', 'COUNTRY_CODE', 'LAUNCH_DATE', 'SITE',
       'DECAY_DATE', 'FILE', 'GP_ID', 'TLE_LINE0', 'TLE_LINE1', 'TLE_LINE2'],
      dtype='object')

In [206]:
# Define array of satellite objects based on TLE data
sat_arry = SatrecArray([
    Satrec.twoline2rv(t1, t2) for t1, t2 in df[["TLE_LINE1", "TLE_LINE2"]].to_numpy()
])

In [217]:
# Define future timestamps to propagate orbits
diff_min = 5
total_min = 60 * 24
curr_ts = datetime.now(timezone.utc)
future_ts_li = []
for i in range(diff_min, total_min + diff_min, diff_min):
    ts = curr_ts + timedelta(minutes=i)
    future_ts_li.append(ts)

# Convert timestamps to Julian dates
jd_arry = fr_arry = np.empty(0)
for ts in future_ts_li:
    jd, fr = jday(
        ts.year,
        ts.month,
        ts.day,
        ts.hour,
        ts.minute,
        ts.second + ts.microsecond / 1e6
    )
    jd_arry = np.append(jd_arry, jd)
    fr_arry = np.append(fr_arry, fr)

# Propagate orbits using future dates
e, r, v = sat_arry.sgp4(jd_arry, fr_arry)  # error, position, velocity

In [ ]:
# Define dict for satellite IDs and their propagated orbits
sat_orbit_dict = dict(zip(df["NORAD_CAT_ID"], r))

In [240]:
from scipy.spatial import KDTree
# For each timestamp, find neighbor pairs
for i, t in enumerate(future_ts_li):
    coords = []
    sat_ids = []
    for sid, pos_list in sat_orbit_dict.items():
        if i < len(pos_list):
            coords.append(pos_list[i])
            sat_ids.append(sid)

    coords = np.array(coords)
    tree = KDTree(coords)
    pairs = []

    for idx, sid in enumerate(sat_ids):
        dists, neighbors = tree.query(coords[idx], k=4)  # 3 nearest neighbors + self
        for j in range(1, len(neighbors)):
            pair = tuple(sorted([sid, sat_ids[neighbors[j]]]))
            pairs.append(pair)

    unique_pairs = list(set(pairs))
    print(f"Time {t}: {len(unique_pairs)} potential close pairs found")

Time 2025-05-11 17:10:41.959538+00:00: 39815 potential close pairs found
Time 2025-05-11 17:15:41.959538+00:00: 39809 potential close pairs found
Time 2025-05-11 17:20:41.959538+00:00: 39807 potential close pairs found
Time 2025-05-11 17:25:41.959538+00:00: 39732 potential close pairs found
Time 2025-05-11 17:30:41.959538+00:00: 39767 potential close pairs found
Time 2025-05-11 17:35:41.959538+00:00: 39914 potential close pairs found
Time 2025-05-11 17:40:41.959538+00:00: 39791 potential close pairs found
Time 2025-05-11 17:45:41.959538+00:00: 39870 potential close pairs found
Time 2025-05-11 17:50:41.959538+00:00: 39802 potential close pairs found
Time 2025-05-11 17:55:41.959538+00:00: 39809 potential close pairs found
Time 2025-05-11 18:00:41.959538+00:00: 39827 potential close pairs found
Time 2025-05-11 18:05:41.959538+00:00: 39828 potential close pairs found
Time 2025-05-11 18:10:41.959538+00:00: 39945 potential close pairs found
Time 2025-05-11 18:15:41.959538+00:00: 39796 potent

KeyboardInterrupt: 

In [239]:
# Check SGP4 errors
from sgp4.api import SGP4_ERRORS
SGP4_ERRORS

{1: 'mean eccentricity is outside the range 0.0 to 1.0',
 2: 'nm is less than zero',
 3: 'perturbed eccentricity is outside the range 0.0 to 1.0',
 4: 'semilatus rectum is less than zero',
 5: '(error 5 no longer in use; it meant the satellite was underground)',
 6: 'mrt is less than 1.0 which indicates the satellite has decayed'}